In [50]:
pip install opencv-contrib-python

Note: you may need to restart the kernel to use updated packages.


In [52]:
pip install deepface

Note: you may need to restart the kernel to use updated packages.


In [54]:
pip install tf-keras

Note: you may need to restart the kernel to use updated packages.


In [55]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [56]:
pip search tensorflow

Note: you may need to restart the kernel to use updated packages.


ERROR: XMLRPC request failed [code: -32500]
RuntimeError: PyPI no longer supports 'pip search' (or XML-RPC search). Please use https://pypi.org/search (via a browser) instead. See https://warehouse.pypa.io/api-reference/xml-rpc.html#deprecated-methods for more information.


In [ ]:

import tensorflow as tf 
import cv2 
import os
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np


from keras.utils import to_categorical

In [73]:
### It will be working in condition but show the only face border

import cv2

# Load the face cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start capturing video from the default camera (0)
video_cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, video_data = video_cap.read()

    # Convert frame to grayscale
    gray_video_data = cv2.cvtColor(video_data, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_video_data, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw rectangles around the detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(video_data, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow("Video Live", video_data)

    # Break the loop if 'a' is pressed
    if cv2.waitKey(10) == ord("a"):
        break

# Release the capture and close all windows
video_cap.release()
cv2.destroyAllWindows()


In [74]:
# trying to save the new faces in lib

import cv2

# Load the face cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start capturing video from the default camera (0)
video_cap = cv2.VideoCapture(0)

# List to store previously detected faces
prev_faces = []

# Counter for naming captured face images
face_counter = 1

while True:
    # Capture frame-by-frame
    ret, video_data = video_cap.read()

    # Convert frame to grayscale
    gray_video_data = cv2.cvtColor(video_data, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_video_data, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw rectangles around the detected faces
    for (x, y, w, h) in faces:
        # Check if the current face is in the list of previously detected faces
        if (x, y, w, h) in prev_faces:
            # If the face is old, draw a green rectangle
            color = (0, 255, 0)
        else:
            # If the face is new, draw a red rectangle
            color = (0, 0, 255)
            # Save the captured face image
            cv2.imwrite(f'face_{face_counter}.jpg', video_data[y:y+h, x:x+w])
            face_counter += 1

        cv2.rectangle(video_data, (x, y), (x+w, y+h), color, 2)

    # Update the list of previously detected faces
    prev_faces = faces

    # Display the resulting frame
    cv2.imshow("Video Live", video_data)

    # Break the loop if 'a' is pressed
    if cv2.waitKey(10) == ord("a"):
        break

# Release the capture and close all windows
video_cap.release()
cv2.destroyAllWindows()


In [78]:
# only run with the neutral emotion

import cv2

# Load the pre-trained Haar Cascade classifier for face detection from cv2 lib 
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the pre-trained Haar Cascade classifier for eye detection from cv2
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Load the pre-trained Haar Cascade classifier for smile detection from cv2
smile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')

# Load the pre-trained model for facial emotion recognition
# You can replace this with any other model you have for emotion recognition
# This example uses a dummy function to simulate emotion recognition
def detect_emotion(face_roi):
    # Dummy function, replace this with your actual emotion recognition model
    return "Neutral"

# Function to detect faces and emotions
def detect_faces_and_emotions(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        
        # Detect eyes
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color, (ex,ey), (ex+ew,ey+eh), (0,255,0), 2)
        
        # Detect smiles
        smiles = smile_cascade.detectMultiScale(roi_gray, scaleFactor=1.8, minNeighbors=20)
        for (sx,sy,sw,sh) in smiles:
            cv2.rectangle(roi_color, (sx,sy), (sx+sw,sy+sh), (0,255,0), 2)
            
            # Crop the smile region and detect emotion
            smile_roi = roi_color[sy:sy+sh, sx:sx+sw]
            emotion = detect_emotion(smile_roi)
            cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    
    return frame

# Capture video from the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = detect_faces_and_emotions(frame)
    
    cv2.imshow('Face Emotion Detection', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('a'):
        break

cap.release()
cv2.destroyAllWindows()


## final project With done all emotion detection

In [81]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define emotions
EMOTIONS = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]

# Load dataset
data = []
labels = []
for emotion_index, emotion in enumerate(EMOTIONS):
    folder_path = f"C:/Users/Laxman Suryawanshi/OneDrive/Desktop/train/{emotion}"
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (48, 48))
        data.append(img)
        labels.append(emotion_index)

# Convert lists to numpy arrays
data = np.array(data, dtype=np.float32)
labels = np.array(labels)

# Normalize data
data /= 255.0

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(EMOTIONS), activation='softmax')
])

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train[..., np.newaxis], y_train, epochs=10, batch_size=32, validation_data=(X_val[..., np.newaxis], y_val))


Epoch 1/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 72s 96ms/step - accuracy: 0.2344 - loss: 1.8332 - val_accuracy: 0.3723 - val_loss: 1.6105
Epoch 2/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 76s 87ms/step - accuracy: 0.3649 - loss: 1.6191 - val_accuracy: 0.4451 - val_loss: 1.4404
Epoch 3/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 52s 72ms/step - accuracy: 0.4414 - loss: 1.4593 - val_accuracy: 0.4761 - val_loss: 1.3542
Epoch 4/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 75s 105ms/step - accuracy: 0.4747 - loss: 1.3752 - val_accuracy: 0.5038 - val_loss: 1.2918
Epoch 5/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 84s 106ms/step - accuracy: 0.4966 - loss: 1.3165 - val_accuracy: 0.5096 - val_loss: 1.2814
Epoch 6/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 78s 100ms/step - accuracy: 0.5221 - loss: 1.2728 - val_accuracy: 0.5195 - val_loss: 1.2456
Epoch 7/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 73s 102ms/step - accuracy: 0.5505 - loss: 1.1828 - val_accuracy: 0.5307 - val_loss: 1.2209
Epoch 9/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 81s 106ms/step - accuracy: 0.5811 - loss: 1.11

In [40]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Load images and labels
data = []
labels = []

emotions_folder = "C:/Users/Laxman Suryawanshi/OneDrive/Desktop/train"
for emotion_index, emotion in enumerate(os.listdir(emotions_folder)):
    emotion_folder = os.path.join(emotions_folder, emotion)
    for img_name in os.listdir(emotion_folder):
        img_path = os.path.join(emotion_folder, img_name)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        img = cv2.resize(img, (48, 48))  # Resize to a fixed size
        data.append(img)
        labels.append(emotion_index)

# Convert lists to numpy arrays
data = np.array(data, dtype=np.float32)
labels = np.array(labels)

# Normalize pixel values to the range [0, 1]
data /= 255.0

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(7, activation='softmax')  # 7 output classes for 7 emotions
])

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train[..., np.newaxis], y_train, epochs=10, batch_size=32, validation_data=(X_val[..., np.newaxis], y_val))


Epoch 1/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - accuracy: 0.3015 - loss: 1.7405 - val_accuracy: 0.4103 - val_loss: 1.5080
Epoch 2/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.4537 - loss: 1.4318 - val_accuracy: 0.4587 - val_loss: 1.4087
Epoch 3/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.5068 - loss: 1.3111 - val_accuracy: 0.4868 - val_loss: 1.3396
Epoch 4/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.5508 - loss: 1.1955 - val_accuracy: 0.5005 - val_loss: 1.3122
Epoch 5/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.5844 - loss: 1.1060 - val_accuracy: 0.5132 - val_loss: 1.3182
Epoch 6/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.6212 - loss: 1.0172 - val_accuracy: 0.5047 - val_loss: 1.3224
Epoch 7/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.6700 - loss: 0.8998 - val_accuracy: 0.5153 - val_loss: 1.3516
Epoch 8/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.7041 - loss: 0.8050 - 

In [87]:
import cv2
import numpy as np

# Load the trained model
#model = model

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Define emotions
EMOTIONS = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]

# Function to detect faces and emotions
def detect_faces_and_emotions(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        face_roi = gray[y:y+h, x:x+w]
        face_roi = cv2.resize(face_roi, (48, 48))
        face_roi = np.expand_dims(face_roi, axis=0)
        face_roi = np.expand_dims(face_roi, axis=3)
        
        # Predict emotion
        prediction = model.predict(face_roi)
        emotion = EMOTIONS[np.argmax(prediction)]
        
        # Draw bounding box around the face and display emotion
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
    
    return frame

# Capture video from the webcamq
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = detect_faces_and_emotions(frame)
    
    cv2.imshow('Real-time Emotion Detection', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('a'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━